In [1]:
import scipy.io as sio
import numpy as np
import tensorly as tl
from tensorly.decomposition import tucker
import pandas as pd

Using numpy backend.


In [2]:
data = sio.loadmat('dnations')
data = data['R']
data.shape

(14, 14, 56)

In [3]:
data[np.isnan(data)] = 0
data[0,0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

## Tucker decomposition to get initial value of G, A, B, C

$$
Y \longrightarrow 10(2Y-1)  = \hat{Y}
$$ 
Thus, we have:$$
\hat{Y}^{14*14*56} = G^{4*4*5} \times_1 A^{14*4}\times_2 B^{14*4} \times_3C^{56*5}
$$

### just draft 

In [ ]:
data = 10*(2*data - 1)
data[0,0]

In [ ]:
core, factors = tucker(data,ranks=(4,4,5))
factors[0].shape

In [ ]:
tl.tenalg.multi_mode_dot(core,factors[:2]).shape

In [ ]:
tl.tenalg.mode_dot(core,factors[1],1).shape

### just draft 

## Matrix Form GLM

Consider 
$$
logit[E(Y))_{n*p}] = U_{n*p} = X_{n*R} \times \beta_{R*p}
$$
Where $U_{n*p} = (u_1,\ldots,u_p)$, $\beta_{R*p} = (\beta_1,\ldots, \beta_p)$

We have:
\begin{align*}
\begin{array}{c}
   u_1^{N*1} =  X^{n*R}\times \beta_1^{R*1} \\ 
   u_2^{N*1} =  X^{n*R}\times \beta_2^{R*1} \\ 
   \vdots\\
   u_p^{N*1} =  X^{n*R}\times \beta_P^{R*1}\\ 
\end{array}
\end{align*}
Then we implement GLM.

Then we use function to realize it:

In [4]:
from sklearn.linear_model import LogisticRegression
def glm_mat(U,X):
    p = U.shape[1]               # The columns of U i.e the number for X to replicate
    R = X.shape[1]               # The number of elements in each columns of beta_i (rows in beta)
    beta = np.arange(R).reshape(R,1)         ## initial beta_i to concatenate
    #print(beta)
    for i in np.arange(p):
        model = LogisticRegression(solver = 'liblinear', multi_class = 'ovr',fit_intercept = False)
        model.fit(X, U[:,i])
        beta_i = model.coef_.reshape(R,1)          ### beta_i : a R*1 vector
        #print(beta_i)
        beta = np.concatenate((beta, beta_i),axis = 1)
    ## Now beta is a R*(p+1) matrix with first column need to drop
    beta = beta[:,1:p+1]                ### drop initial column
    return beta  

In [5]:
b = np.array([1,0,0,1,1,0]).reshape(3,2)
b

array([[1, 0],
       [0, 1],
       [1, 0]])

In [6]:
c = np.arange(9).reshape(3,3)
c

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [7]:
glm_mat(b,c)

array([[-0.06988849,  0.06988849],
       [ 0.02955993, -0.02955993],
       [ 0.12900836, -0.12900836]])

## Tranfer to matrix product form

$$
Y^{14*14*56} = G^{4*4*5} \times_1 A^{14*4}\times_2 B^{14*4} \times_3C^{56*5} = G_{AB}^{14*14*5} \times_3 C^{56*5} \\
$$
Update A,B,C through:
$$
Y_{(3)}^T =  G_{AB(3)}^T C^T
$$
Since for distinct modes in a series of multiplications, the order of the multiplication is irrelevant. Thus, A,B,C the same update way.

Update G through:
$$
Y_{(3)} = C G_{AB(3)} \\
Y_{new raw} = G_{AB(new)}^{14*14*5} \times_3  C^{5*56} \\
$$
Here $Y_{new raw}$ is the likelihood before softmax, get G_new through tucker decomposition.




In [29]:
def update_binary(data,core_shape, Nsim):
    data1 = 10*(2*data - 1)
    core, factors = tucker(data1,ranks=core_shape)
    Y_1 = tl.unfold(data,mode = 0)
    Y_2 = tl.unfold(data,mode = 1)
    Y_3 = tl.unfold(data,mode = 2)        ## get three unfold matrix
    A = factors[0]   ## 14*4   d1*r1
    B = factors[1]   ## 14*4   d2*r2
    C = factors[2]   ## 56*5   d3*r3
    G = core
    d1 = data.shape[0] ; d2 = data.shape[1] ; d3 = data.shape[2]  
    r1 = core_shape[0] ; r2 = core_shape[1] ; r3 = core_shape[2] 
    
    ### kind of a flaw  since Y_3 some columns are all zero
    Y_3[0,np.where(Y_3.sum(0) == 0)] = 1
    ###
    
    for i in np.arange(Nsim):
        G_AB = tl.tenalg.multi_mode_dot(G,factors[:2],modes = [0,1])     ##  14*14*5    d1,d2,r3
        G_AC = tl.tenalg.multi_mode_dot(G,factors[0::2],modes = [0,2])   ## 14*4*56     d1,r2,d3
        G_BC = tl.tenalg.multi_mode_dot(G,factors[1:],modes = [1,2])     ## 4*14*56     r1,d2,d3
        
        ## update A
        G_BC_1 = tl.unfold(G_BC,mode = 0)
        #print(G_BC.shape)
        A = glm_mat(Y_1.transpose(),G_BC_1.transpose()).transpose()
        
        ## update B
        G_AC_2 = tl.unfold(G_AC,mode = 1)
        #print(Y_2.shape)
        B = glm_mat(Y_2.transpose(),G_AC_2.transpose()).transpose()
    
        ## update C
        G_AB_3 = tl.unfold(G_AB,mode = 2)
        C = glm_mat(Y_3.transpose(),G_AB_3.transpose()).transpose()
        
        ## update G
        #print(Y_3[:1])
        G_AB_3_new = glm_mat(Y_3,C)
        G_AB_new = tl.fold(G_AB_3_new,mode = 2,shape = (d1,d2,r3))
        Y_newraw = tl.tenalg.mode_dot(G_AB_new, C,2)
        
        ## in order to update G, we got new likelihood(before softmax) and tucker decomposition again
        ### conduct tucker to obtain new G again
        ### keypoint: tucker decomposition is unique
        G_new, factors = tucker(Y_newraw,ranks=core_shape)
        
        if(np.linalg.norm(G - G_new) < 1):
            print("Done at %d's iteration" % i)
            break
        
        G = G_new

        #A = factors[0]
        #B = factors[1]
        #C = factors[2]
     
    factors = [A,B,C]
    return G,factors    
    

In [31]:
core, membership = update_binary(data,[4,4,5],1000)

In [32]:
print(core.shape,
membership[0].shape)

(4, 4, 5) (14, 4)


##### Another update method:

$$
Y^{14*14*56} = G^{4*4*5} \times_1 A^{14*4}\times_2 B^{14*4} \times_3C^{56*5} = G_{AB}^{14*14*5} \times_3 C^{56*5} \\
$$
Update A,B,C through:
$$
Y_{(3)}^T =  G_{AB(3)}^T C^T
$$
Since for distinct modes in a series of multiplications, the order of the multiplication is irrelevant. Thus, A,B,C the same update way.

Update G through:
$$
Y_{(3)} = C G_{AB(3)} \\
G_{AB(3)} \longrightarrow  G_{AB(new)}
$$

Through G_AB to G_B, we have:
$$
G_{AB(new)}^{14*14*5} = G^{4*4*5} \times_2 B^{14*4} \times_1 A^{14*4} = G_B^{4*14*5} \times_1 A^{14*4}
$$
Thus:
$$
G_{AB(new)(1)}^{14*(14*5)} =  A^{14*4} G_{B(1)}^{4*(14*5)} \\
G_{B(1)} \longrightarrow  G_{B(new)}
$$
We have new $G_{AB(new)(1)}^{14*(14*5)}$ and new $A^{14*4}$, use linear regression to solve $G_{B(1)}^{4*(14*5)}$.

Through G_B to G, we have:
$$
G_{B(new)}^{4*14*5} = G^{4*4*5} \times_2 B^{14*4}
$$

$$
G_{B(new)(2)}^{14*(4*5)} =  B^{14*4} G_{(2)}^{4*(4*5)} \\
G_{(2)} \longrightarrow  G_{(new)}
$$
We have new $G_{B(new)(2)}^{14*(4*5)}$ and new $B^{14*4}$, use linear regression to solve $G_{(2)}^{4*(4*5)}$.



In [58]:
from sklearn.linear_model import LinearRegression

def update_binary(data,core_shape, Nsim):
    data1 = 10*(2*data - 1)
    core, factors = tucker(data1,ranks=core_shape)
    Y_1 = tl.unfold(data,mode = 0)
    Y_2 = tl.unfold(data,mode = 1)
    Y_3 = tl.unfold(data,mode = 2)        ## get three unfold matrix
    A = factors[0]   ## 14*4   d1*r1
    B = factors[1]   ## 14*4   d2*r2
    C = factors[2]   ## 56*5   d3*r3
    G = core
    d1 = data.shape[0] ; d2 = data.shape[1] ; d3 = data.shape[2]  
    r1 = core_shape[0] ; r2 = core_shape[1] ; r3 = core_shape[2] 
    
    ### kind of a flaw  since Y_3 some columns are all zero
    Y_3[0,np.where(Y_3.sum(0) == 0)] = 1
    ###
    
    for i in np.arange(Nsim):
        G_AB = tl.tenalg.multi_mode_dot(G,factors[:2],modes = [0,1])     ##  14*14*5    d1,d2,r3
        G_AC = tl.tenalg.multi_mode_dot(G,factors[0::2],modes = [0,2])   ## 14*4*56     d1,r2,d3
        G_BC = tl.tenalg.multi_mode_dot(G,factors[1:],modes = [1,2])     ## 4*14*56     r1,d2,d3
        
        ## update A
        G_BC_1 = tl.unfold(G_BC,mode = 0)
        #print(G_BC.shape)
        A = glm_mat(Y_1.transpose(),G_BC_1.transpose()).transpose()
        
        ## update B
        G_AC_2 = tl.unfold(G_AC,mode = 1)
        #print(Y_2.shape)
        B = glm_mat(Y_2.transpose(),G_AC_2.transpose()).transpose()
    
        ## update C
        G_AB_3 = tl.unfold(G_AB,mode = 2)
        C = glm_mat(Y_3.transpose(),G_AB_3.transpose()).transpose()
        
        ## update G
        #print(Y_3[:1])
        G_AB_3_new = glm_mat(Y_3,C)
        G_AB_new = tl.fold(G_AB_3_new,mode = 2,shape = (d1,d2,r3))
        
        #### through G_AB to G_B
        G_AB_new_1 = tl.unfold(G_AB_new,mode = 0)
        model = LinearRegression(fit_intercept = False)
        G_B_1_new = model.fit(A,G_AB_new_1).coef_.transpose()    ## G_B_1_new r1*(d2*r3)
        G_B_new = tl.fold(G_B_1_new,mode = 0,shape = (r1,d2,r3))
        
        #### through G_B to G
        G_B_new_2 = tl.unfold(G_B_new,mode = 1)
        model = LinearRegression(fit_intercept = False)
        G_2_new = model.fit(B,G_B_new_2).coef_.transpose()    ## G_2_new r2*(r1*r3)
        G_new = tl.fold(G_2_new, mode = 1, shape = (r1,r2,r3))
        
        
        
        if(np.linalg.norm(G - G_new) < 1):
            print("Done at %d's iteration" % i)
            break
        
        G = G_new

        #A = factors[0]
        #B = factors[1]
        #C = factors[2]
     
    factors = [A,B,C]
    return G,factors    
    

In [59]:
core, membership = update_binary(data,[4,4,5],50)

In [ ]:
print(core.shape,membership[0].shape)

## Kmeans to cluster

In [33]:
from sklearn.cluster import KMeans
km0 = KMeans(n_clusters=4, random_state=0)
km0.fit(membership[0])

km1 = KMeans(n_clusters=4, random_state=0)
km1.fit(membership[1])

km2 = KMeans(n_clusters=5, random_state=0)
km2.fit(membership[2])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [34]:
pd.get_dummies(km0.labels_)

,0,1,2,3
0,0,0,1,0
1,1,0,0,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0
5,0,0,0,1
6,1,0,0,0
7,0,0,0,1
8,1,0,0,0
9,0,0,1,0


In [35]:
pd.get_dummies(km1.labels_)

,0,1,2,3
0,0,0,1,0
1,0,1,0,0
2,0,1,0,0
3,1,0,0,0
4,0,1,0,0
5,0,0,0,1
6,0,1,0,0
7,0,0,0,1
8,0,1,0,0
9,0,0,1,0


In [15]:
pd.get_dummies(km2.labels_)

,0,1,2,3,4
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,1,0,0,0,0
9,1,0,0,0,0


In [16]:
def sig(x):
    return 1/ np.exp(core)

In [17]:
sig(core)

array([[[2.14244058e+55, 7.53897912e-02, 1.14319376e+00, 3.44326124e-01,
         1.79875227e+00],
        [2.17330643e-02, 2.50911057e-14, 3.50685241e-11, 6.36529794e-01,
         1.09301800e+02],
        [4.63527763e-01, 7.77179447e-05, 9.24014334e+06, 3.24942642e-02,
         5.47089965e+00],
        [4.53733277e+00, 4.97139107e+01, 8.93388354e+02, 3.34235512e-01,
         7.94519059e+00]],

       [[1.08208242e+01, 2.24652346e+09, 9.00799637e-07, 2.98722472e+03,
         8.14797224e+06],
        [3.16097073e+00, 5.69589256e-03, 1.54970551e+02, 6.88745418e+09,
         2.20795573e+01],
        [1.32120259e+02, 1.03739600e+03, 7.61118753e-03, 1.19667439e+07,
         3.30644940e-05],
        [2.56911364e-01, 1.01944420e-04, 1.99826675e-01, 4.93083324e+01,
         2.12694757e-03]],

       [[1.87883385e+00, 1.04977105e+03, 5.32523186e-03, 2.58650924e-01,
         7.51850427e-06],
        [3.57538136e+02, 5.00049483e+05, 9.96810663e-06, 1.96289851e+00,
         2.14697503e-05],
      

In [18]:
np.linalg.norm(core)

148.14142302479036